# Bert Analysis of NLP Disaster Tweets


## Important Installations and imports

In [31]:
!pip install transformers

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers as tfm

# Configuring  TPU 




In [33]:
AUTO = tf.data.experimental.AUTOTUNE

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.97.79.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.97.79.170:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Helper Functions for encoding the input and building the model


In [34]:
def encode(texts, tokenizer, max_len=64):
    tokens = tokenizer.batch_encode_plus(
        texts, max_length=max_len, pad_to_max_length=True
    )
    return np.array(tokens['input_ids'])

In [35]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

We use Adam optimizer to trackback our losses.Then we use BinaryCrossentropy to calculate our losses. And use the accuracy metric, and then we finally complie our model.

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
EPOCHS = 10
MAX_LEN = 64
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

# Load data

In [38]:
train = pd.read_csv("/content/drive/My Drive/CMPE-257/NLPDataset/train.csv")
test = pd.read_csv("/content/drive/My Drive/CMPE-257/NLPDataset/test.csv")
submission = pd.read_csv("/content/drive/My Drive/CMPE-257/NLPDataset/sample_submission.csv")

In [39]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [40]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [41]:
submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [42]:
train, valid = train_test_split(train, train_size=0.8, random_state=17)
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-large-uncased')

#Tokenize Data 

In [45]:
tokenizer = tfm.BertTokenizerFast.from_pretrained('bert-large-uncased')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [57]:
train_input = encode(train.text.tolist(), tokenizer, max_len=84)
valid_input = encode(valid.text.tolist(), tokenizer, max_len=84)
test_input = encode(test.text.tolist(), tokenizer, max_len=84)

train_labels = train.target.values
valid_labels = valid.target.values


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [47]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_input, train_labels))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)


In [56]:
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_input, valid_labels))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

# Load model and train the model 

We have already imported the BERT model and it's respective tokenizer.

The bert models can be found from here : https://huggingface.co/models We are using the 'bert-large-uncased' model here. But more models can be found in the above links.

'bert-large-uncased' is the exact bert model we are going to use. We use AutoTokenizer and it will create a generic tokenizer class that will be instantiated as one of the tokenizer classes of the hugging face library when created with the AutoTokenizer.from_pretrained() class method. We can pass the BERT model of our choice through it to get the respective tokenizers.

In [50]:
%%time
with strategy.scope():
    transformer_layer = tfm.TFBertModel.from_pretrained( 'bert-large-uncased')
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 64)]             0         
                                                                 
 tf_bert_model_1 (TFBertMode  TFBaseModelOutputWithPoo  335141888
 l)                          lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             64, 1024),                          
                              pooler_output=(None, 10            
                             24),                                
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [51]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 64)]             0         
                                                                 
 tf_bert_model_1 (TFBertMode  TFBaseModelOutputWithPoo  335141888
 l)                          lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             64, 1024),                          
                              pooler_output=(None, 10            
                             24),                                
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

In [52]:
N_STEPS = train_input.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=N_STEPS,
    epochs=EPOCHS,
    validation_data=valid_dataset
    )

Epoch 1/10


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 64) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>]


Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 64) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=int64>]


47/47 [==============================] - ETA: 0s - loss: 0.4997 - accuracy: 0.7631

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 64) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None,) dtype=int64>]


47/47 [==============================] - 204s 930ms/step - loss: 0.4997 - accuracy: 0.7631 - val_loss: 0.3966 - val_accuracy: 0.8240
Epoch 2/10
47/47 [==============================] - 14s 306ms/step - loss: 0.3834 - accuracy: 0.8404 - val_loss: 0.3984 - val_accuracy: 0.8352
Epoch 3/10
47/47 [==============================] - 14s 302ms/step - loss: 0.3538 - accuracy: 0.8569 - val_loss: 0.4073 - val_accuracy: 0.8411
Epoch 4/10
47/47 [==============================] - 15s 318ms/step - loss: 0.3040 - accuracy: 0.8810 - val_loss: 0.3906 - val_accuracy: 0.8404
Epoch 5/10
47/47 [==============================] - 14s 305ms/step - loss: 0.2714 - accuracy: 0.8968 - val_loss: 0.4365 - val_accuracy: 0.8339
Epoch 6/10
47/47 [==============================] - 14s 303ms/step - loss: 0.2461 - accuracy: 0.9039 - val_loss: 0.4355 - val_accuracy: 0.8299
Epoch 7/10
47/47 [==============================] - 14s 303ms/step - loss: 0.1994 - accuracy: 0.9249 - val_loss: 0.5145 - val_accuracy: 0.8319
Epoch 8/1

# Predict and Submit

In [53]:
test_pred = model.predict(test_input, verbose=1, batch_size=BATCH_SIZE)
submission['target'] = test_pred.round().astype(int)


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 64) dtype=int64>]


26/26 [==============================] - 31s 925ms/step


In [54]:
submission.to_csv('/content/drive/My Drive/CMPE-257/NLPDataset/submission.csv', index=False)

In [55]:
submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
